In [12]:
from PIL import Image
import os, glob, random
import SimpleITK as sitk
import numpy as np
import cv2
import pandas as pd


# data_dir = 'videos/AFL_MRI/data_in_nii'

# img_nii_dir = os.path.join(data_dir, 'AFL_img_full_body_45')
# seg_nii_dir =  os.path.join(data_dir, 'AFL_label_segmentation_full_body')

# img_png_dir = os.path.join(data_dir, 'img_in_jpg_AFL_L4L5_4class')
# seg_png_dir = os.path.join(data_dir, 'label_in_png_AFL_L4L5_4class')

# csv_path = os.path.join(data_dir, 'AFL_img_45_L2-L5_annotation.csv')


data_dir = 'videos/bed_rest_MRI/data_in_nii'

img_nii_dir = os.path.join(data_dir, 'img_in_nii_72')
seg_nii_dir =  os.path.join(data_dir, 'label_in_nii_72')

img_png_dir = os.path.join(data_dir, 'img_in_jpg_L4L5_4class')
seg_png_dir = os.path.join(data_dir, 'label_in_png_L4L5_4class')

csv_path = os.path.join(data_dir, 'Bedrest_img_72_L4L5_annotation.csv')



# 读取CSV文件
df = pd.read_csv(csv_path )

# 逐行打印不同的值
for index, row in df.iterrows():
#     print
    slice_stop  = int(row['L4_L5']) 
#     print(index, row)
    d = row['img_name']
    pid =  d.split(('_'))[0] + '_' +d.split(('_'))[1]
    
    if 'West_Bd' in d or 'N_Melb' in d:
        pid =  d.split(('_'))[0] + '_' +d.split(('_'))[1]+ '_' +d.split(('_'))[2]
        
    print(index, pid, slice_stop)
    img_path = glob.glob(os.path.join(img_nii_dir, pid + "*.nii.gz"))[0]
    seg_path = glob.glob(os.path.join(seg_nii_dir, pid + "*.nii.gz"))[0]
    
    img_sitk = sitk.ReadImage(img_path)
    img_array =  sitk.GetArrayFromImage(img_sitk)
    seg_sitk = sitk.ReadImage(seg_path)
    seg_array =  sitk.GetArrayFromImage(seg_sitk) 
#     seg_array[seg_array > 2] = 0

#     seg_array[seg_array == 2] = 1
#     print(img_array.shape, seg_array.shape)
    img_slice = img_array[ slice_stop , :, :]
    seg_slice = seg_array[slice_stop, :, :]
#     print(img_array.shape, seg_array.shape)


     # Compute the 0.5th and 99.5th percentiles
    lower_bound = np.percentile(img_slice, 0.5)
    upper_bound = np.percentile(img_slice, 99.5)

    # Clip the intensity values to the range [lower_bound, upper_bound]
    img_slice = np.clip(img_slice, lower_bound, upper_bound)

    if 'ID' in pid:
        img_slice =np.rot90(img_slice, 2) 
        seg_slice =np.rot90(seg_slice, 2) 
    # Rescale the intensity values to the range [0, 255]
    img_slice = (img_slice - lower_bound) / (upper_bound - lower_bound)
    img_slice = (img_slice * 255).astype(np.uint8)

    # Convert the image and segmentation slices to PIL Image format
    img_pil = Image.fromarray(img_slice)
#             seg_pil = Image.fromarray((255- seg_slice * 255).astype(np.uint8))
    seg_pil = Image.fromarray((seg_slice  * 50 ).astype(np.uint8))

    # Create the output directories if they do not exist
    os.makedirs(img_png_dir, exist_ok=True)
    os.makedirs(seg_png_dir, exist_ok=True)

    # Save the slices as PNG files

    img_output_path = os.path.join(img_png_dir, f'{pid}_slice_{slice_stop:03d}_img.jpg')
    seg_output_path = os.path.join(seg_png_dir, f'{pid}_slice_{slice_stop:03d}_seg.png')

    img_pil.save(img_output_path)
    seg_pil.save(seg_output_path)

    print(f"Saved image slice: {img_output_path}")
    print(f"Saved segmentation slice: {seg_output_path}")


0 ID9J_R13 39
Saved image slice: videos/bed_rest_MRI/data_in_nii/img_in_jpg_L4L5_4class/ID9J_R13_slice_039_img.jpg
Saved segmentation slice: videos/bed_rest_MRI/data_in_nii/label_in_png_L4L5_4class/ID9J_R13_slice_039_seg.png
1 ID9J_R0 23
Saved image slice: videos/bed_rest_MRI/data_in_nii/img_in_jpg_L4L5_4class/ID9J_R0_slice_023_img.jpg
Saved segmentation slice: videos/bed_rest_MRI/data_in_nii/label_in_png_L4L5_4class/ID9J_R0_slice_023_seg.png
2 ID9J_BDC 27
Saved image slice: videos/bed_rest_MRI/data_in_nii/img_in_jpg_L4L5_4class/ID9J_BDC_slice_027_img.jpg
Saved segmentation slice: videos/bed_rest_MRI/data_in_nii/label_in_png_L4L5_4class/ID9J_BDC_slice_027_seg.png
3 ID8H_R13 28
Saved image slice: videos/bed_rest_MRI/data_in_nii/img_in_jpg_L4L5_4class/ID8H_R13_slice_028_img.jpg
Saved segmentation slice: videos/bed_rest_MRI/data_in_nii/label_in_png_L4L5_4class/ID8H_R13_slice_028_seg.png
4 ID8H_R0 24
Saved image slice: videos/bed_rest_MRI/data_in_nii/img_in_jpg_L4L5_4class/ID8H_R0_slice_02

Saved image slice: videos/bed_rest_MRI/data_in_nii/img_in_jpg_L4L5_4class/ID20V_R0_slice_025_img.jpg
Saved segmentation slice: videos/bed_rest_MRI/data_in_nii/label_in_png_L4L5_4class/ID20V_R0_slice_025_seg.png
38 ID20V_BDC 38
Saved image slice: videos/bed_rest_MRI/data_in_nii/img_in_jpg_L4L5_4class/ID20V_BDC_slice_038_img.jpg
Saved segmentation slice: videos/bed_rest_MRI/data_in_nii/label_in_png_L4L5_4class/ID20V_BDC_slice_038_seg.png
39 ID1A_R13 20
Saved image slice: videos/bed_rest_MRI/data_in_nii/img_in_jpg_L4L5_4class/ID1A_R13_slice_020_img.jpg
Saved segmentation slice: videos/bed_rest_MRI/data_in_nii/label_in_png_L4L5_4class/ID1A_R13_slice_020_seg.png
40 ID1A_R0 17
Saved image slice: videos/bed_rest_MRI/data_in_nii/img_in_jpg_L4L5_4class/ID1A_R0_slice_017_img.jpg
Saved segmentation slice: videos/bed_rest_MRI/data_in_nii/label_in_png_L4L5_4class/ID1A_R0_slice_017_seg.png
41 ID1A_BDC 15
Saved image slice: videos/bed_rest_MRI/data_in_nii/img_in_jpg_L4L5_4class/ID1A_BDC_slice_015_img

In [2]:
from PIL import Image
import os, glob, random
import SimpleITK as sitk
import numpy as np
import cv2
import pandas as pd

data_dir = 'videos/bed_rest_MRI'
img_nii_dir = os.path.join(data_dir, 'img_in_nii_24')
seg_nii_dir =  os.path.join(data_dir, 'label_in_nii_24')

img_png_dir = os.path.join(data_dir, 'Bedrest_L4L5_in_JPG_2class_DINO_1', 'img_in_jpg_L4L5')
seg_png_dir = os.path.join(data_dir, 'Bedrest_L4L5_in_JPG_2class_DINO_1','label_in_png_L4L5')


csv_path = os.path.join(data_dir, 'bedrest_bdc_22_L2-L5_annotation.csv')

# 读取CSV文件
df = pd.read_csv(csv_path )

# 逐行打印不同的值
for index, row in df.iterrows():
    
#     print
    
    slice_stop  = int(row['L4_L5']) 
    d = row['img_name']
    pid =  d.split(('_'))[0] + '_' +d.split(('_'))[1]  
    
    
        
    print(index, pid, slice_stop)
    img_path = glob.glob(os.path.join(img_nii_dir, pid + "*.nii.gz"))[0]
    seg_path = glob.glob(os.path.join(seg_nii_dir, pid + "*.nii.gz"))[0]

    img_sitk = sitk.ReadImage(img_path)
    img_array =  sitk.GetArrayFromImage(img_sitk)
    seg_sitk = sitk.ReadImage(seg_path)
    seg_array =  sitk.GetArrayFromImage(seg_sitk) 
#     seg_array[seg_array > 2] = 0

#     seg_array[seg_array == 2] = 1
#     print(img_array.shape, seg_array.shape)
    img_slice = img_array[slice_stop , :, :]
    seg_slice = seg_array[slice_stop, :, :]
#     print(img_array.shape, seg_array.shape)
    if 'ID' in pid:
        img_slice =np.rot90(img_slice, 2) 
        seg_slice =np.rot90(seg_slice, 2) 
    unique_values = np.unique(seg_slice )
    print('unique value in seg_array',unique_values )
     # Compute the 0.5th and 99.5th percentiles
    lower_bound = np.percentile(img_slice, 0.5)
    upper_bound = np.percentile(img_slice, 99.5)

    # Clip the intensity values to the range [lower_bound, upper_bound]
    img_slice = np.clip(img_slice, lower_bound, upper_bound)


    # Rescale the intensity values to the range [0, 255]
    img_slice = (img_slice - lower_bound) / (upper_bound - lower_bound)
    img_slice = (img_slice * 255).astype(np.uint8)

    # Convert the image and segmentation slices to PIL Image format
    img_pil = Image.fromarray(img_slice)
#             seg_pil = Image.fromarray((255- seg_slice * 255).astype(np.uint8))
    seg_pil = Image.fromarray((seg_slice * 50).astype(np.uint8))

    # Create the output directories if they do not exist
    os.makedirs(img_png_dir, exist_ok=True)
    os.makedirs(seg_png_dir, exist_ok=True)

    # Save the slices as PNG files

    img_output_path = os.path.join(img_png_dir, f'{pid}_slice_{slice_stop:03d}_img_.jpg')
    seg_output_path = os.path.join(seg_png_dir, f'{pid}_slice_{slice_stop:03d}_seg_.png')

    img_pil.save(img_output_path)
    seg_pil.save(seg_output_path)

    print(f"Saved image slice: {img_output_path}")
    print(f"Saved segmentation slice: {seg_output_path}")


0 ID8H_BDC 20
unique value in seg_array [0. 1. 2.]
Saved image slice: videos/bed_rest_MRI/Bedrest_L4L5_in_JPG_2class_DINO_1/img_in_jpg_L4L5/ID8H_BDC_slice_020_img_.jpg
Saved segmentation slice: videos/bed_rest_MRI/Bedrest_L4L5_in_JPG_2class_DINO_1/label_in_png_L4L5/ID8H_BDC_slice_020_seg_.png
1 ID7G_BDC 22
unique value in seg_array [0. 1. 2.]
Saved image slice: videos/bed_rest_MRI/Bedrest_L4L5_in_JPG_2class_DINO_1/img_in_jpg_L4L5/ID7G_BDC_slice_022_img_.jpg
Saved segmentation slice: videos/bed_rest_MRI/Bedrest_L4L5_in_JPG_2class_DINO_1/label_in_png_L4L5/ID7G_BDC_slice_022_seg_.png
2 ID6F_BDC 17
unique value in seg_array [0. 1. 2.]
Saved image slice: videos/bed_rest_MRI/Bedrest_L4L5_in_JPG_2class_DINO_1/img_in_jpg_L4L5/ID6F_BDC_slice_017_img_.jpg
Saved segmentation slice: videos/bed_rest_MRI/Bedrest_L4L5_in_JPG_2class_DINO_1/label_in_png_L4L5/ID6F_BDC_slice_017_seg_.png
3 ID5E_BDC 18
unique value in seg_array [0. 1. 2.]
Saved image slice: videos/bed_rest_MRI/Bedrest_L4L5_in_JPG_2class_D

In [5]:
from PIL import Image
import os, glob, random
import SimpleITK as sitk
import numpy as np
import cv2
import pandas as pd


data_dir = 'videos/TUM_MRI'
img_nii_dir = os.path.join(data_dir, 'img_in_nii_full')
seg_nii_dir =  os.path.join(data_dir, 'label_in_nii_full')

img_png_dir = os.path.join(data_dir, 'TUM_L4L5_in_JPG_2class_DINO_1/img_in_jpg_L4L5')
seg_png_dir = os.path.join(data_dir, 'TUM_L4L5_in_JPG_2class_DINO_1/label_in_png_L4L5')


csv_path = os.path.join(data_dir, 'MyoSegmenTUM_54_water_fat_L2-L5.csv')


# 读取CSV文件
df = pd.read_csv(csv_path )

# 逐行打印不同的值
for index, row in df.iterrows():
#     print
    slice_stop  = int(row['L4_L5']) 
    if slice_stop >0:
        d = row['img_name']
        pid =  d.split(('_'))[0] + '_' +d.split(('_'))[1]

        if 'West_Bd' in d or 'N_Melb' in d:
            pid =  d.split(('_'))[0] + '_' +d.split(('_'))[1]+ '_' +d.split(('_'))[2]

        print(index, pid, slice_stop)
        img_path = glob.glob(os.path.join(img_nii_dir, pid + "*.nii.gz"))[0]
        seg_path = glob.glob(os.path.join(seg_nii_dir, 'TUM' + pid + "*.nii.gz"))[0]

        img_sitk = sitk.ReadImage(img_path)
        img_array =  sitk.GetArrayFromImage(img_sitk)
        seg_sitk = sitk.ReadImage(seg_path)
        seg_array =  sitk.GetArrayFromImage(seg_sitk) 
        seg_array[seg_array > 2] = 0

    #     seg_array[seg_array == 2] = 1
    #     print(img_array.shape, seg_array.shape)
        img_slice = img_array[ slice_stop , :, :]
        seg_slice = seg_array[slice_stop, :, :]
    #     print(img_array.shape, seg_array.shape)


         # Compute the 0.5th and 99.5th percentiles
        lower_bound = np.percentile(img_slice, 0.5)
        upper_bound = np.percentile(img_slice, 99.5)

        # Clip the intensity values to the range [lower_bound, upper_bound]
        img_slice = np.clip(img_slice, lower_bound, upper_bound)


        # Rescale the intensity values to the range [0, 255]
        img_slice = (img_slice - lower_bound) / (upper_bound - lower_bound)
        img_slice = (img_slice * 255).astype(np.uint8)

        # Convert the image and segmentation slices to PIL Image format
        img_pil = Image.fromarray(img_slice)
    #             seg_pil = Image.fromarray((255- seg_slice * 255).astype(np.uint8))
        seg_pil = Image.fromarray((seg_slice  * 50 ).astype(np.uint8))

        # Create the output directories if they do not exist
        os.makedirs(img_png_dir, exist_ok=True)
        os.makedirs(seg_png_dir, exist_ok=True)

        # Save the slices as PNG files

        img_output_path = os.path.join(img_png_dir, f'{pid}_slice_{slice_stop:03d}_img_.jpg')
        seg_output_path = os.path.join(seg_png_dir, f'{pid}_slice_{slice_stop:03d}_seg_.png')

        img_pil.save(img_output_path)
        seg_pil.save(seg_output_path)

        print(f"Saved image slice: {img_output_path}")
        print(f"Saved segmentation slice: {seg_output_path}")


0 01_water 2
Saved image slice: videos/TUM_MRI/TUM_L4L5_in_JPG_2class_DINO_1/img_in_jpg_L4L5/01_water_slice_002_img_.jpg
Saved segmentation slice: videos/TUM_MRI/TUM_L4L5_in_JPG_2class_DINO_1/label_in_png_L4L5/01_water_slice_002_seg_.png
1 02_water 15
Saved image slice: videos/TUM_MRI/TUM_L4L5_in_JPG_2class_DINO_1/img_in_jpg_L4L5/02_water_slice_015_img_.jpg
Saved segmentation slice: videos/TUM_MRI/TUM_L4L5_in_JPG_2class_DINO_1/label_in_png_L4L5/02_water_slice_015_seg_.png
2 03_water 27
Saved image slice: videos/TUM_MRI/TUM_L4L5_in_JPG_2class_DINO_1/img_in_jpg_L4L5/03_water_slice_027_img_.jpg
Saved segmentation slice: videos/TUM_MRI/TUM_L4L5_in_JPG_2class_DINO_1/label_in_png_L4L5/03_water_slice_027_seg_.png
3 04_water 17
Saved image slice: videos/TUM_MRI/TUM_L4L5_in_JPG_2class_DINO_1/img_in_jpg_L4L5/04_water_slice_017_img_.jpg
Saved segmentation slice: videos/TUM_MRI/TUM_L4L5_in_JPG_2class_DINO_1/label_in_png_L4L5/04_water_slice_017_seg_.png
4 05_water 18
Saved image slice: videos/TUM_M

In [ ]:
from PIL import Image
import os, glob, random
import SimpleITK as sitk
import numpy as np
import cv2
import pandas as pd


data_dir = 'videos/TUM_MRI'
img_nii_dir = os.path.join(data_dir, 'img_in_nii_full')
seg_nii_dir =  os.path.join(data_dir, 'label_in_nii_full')

img_png_dir = os.path.join(data_dir, 'TUM_L4L5_in_JPG_2class_DINO_1/img_in_jpg_L4L5')
seg_png_dir = os.path.join(data_dir, 'TUM_L4L5_in_JPG_2class_DINO_1/label_in_png_L4L5')


csv_path = os.path.join(data_dir, 'MyoSegmenTUM_54_water_fat_L2-L5.csv')


# 读取CSV文件
df = pd.read_csv(csv_path )

# 逐行打印不同的值
for index, row in df.iterrows():
#     print
    slice_stop  = int(row['L4_L5']) 
    if slice_stop >0:
        d = row['img_name']
        pid =  d.split(('_'))[0] + '_' +d.split(('_'))[1]

        if 'West_Bd' in d or 'N_Melb' in d:
            pid =  d.split(('_'))[0] + '_' +d.split(('_'))[1]+ '_' +d.split(('_'))[2]

        print(index, pid, slice_stop)
        img_path = glob.glob(os.path.join(img_nii_dir, pid + "*.nii.gz"))[0]
        seg_path = glob.glob(os.path.join(seg_nii_dir, 'TUM' + pid + "*.nii.gz"))[0]

        img_sitk = sitk.ReadImage(img_path)
        img_array =  sitk.GetArrayFromImage(img_sitk)
        seg_sitk = sitk.ReadImage(seg_path)
        seg_array =  sitk.GetArrayFromImage(seg_sitk) 
        seg_array[seg_array > 2] = 0

    #     seg_array[seg_array == 2] = 1
    #     print(img_array.shape, seg_array.shape)
        img_slice = img_array[ slice_stop , :, :]
        seg_slice = seg_array[slice_stop, :, :]
    #     print(img_array.shape, seg_array.shape)


         # Compute the 0.5th and 99.5th percentiles
        lower_bound = np.percentile(img_slice, 0.5)
        upper_bound = np.percentile(img_slice, 99.5)

        # Clip the intensity values to the range [lower_bound, upper_bound]
        img_slice = np.clip(img_slice, lower_bound, upper_bound)


        # Rescale the intensity values to the range [0, 255]
        img_slice = (img_slice - lower_bound) / (upper_bound - lower_bound)
        img_slice = (img_slice * 255).astype(np.uint8)

        # Convert the image and segmentation slices to PIL Image format
        img_pil = Image.fromarray(img_slice)
    #             seg_pil = Image.fromarray((255- seg_slice * 255).astype(np.uint8))
        seg_pil = Image.fromarray((seg_slice  * 50 ).astype(np.uint8))

        # Create the output directories if they do not exist
        os.makedirs(img_png_dir, exist_ok=True)
        os.makedirs(seg_png_dir, exist_ok=True)

        # Save the slices as PNG files

        img_output_path = os.path.join(img_png_dir, f'{pid}_slice_{slice_stop:03d}_img_.jpg')
        seg_output_path = os.path.join(seg_png_dir, f'{pid}_slice_{slice_stop:03d}_seg_.png')

        img_pil.save(img_output_path)
        seg_pil.save(seg_output_path)

        print(f"Saved image slice: {img_output_path}")
        print(f"Saved segmentation slice: {seg_output_path}")
